# Bootstrap方法
## I. Bootstrap sample方法的用法示例
### I.1 一个引导案例
实验室有100个学生，想统计他们每人每天要开多少次手机。只有30个学生自愿报名参加统计，安装了用于统计开屏次数的app。1天之后，用他们的数据计算得到平均开屏次数是228.06次。

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# 生成100个学生的总体数据
np.random.seed(42)
pickups = np.random.randint(0,500 , size=100)
print(f"{pickups.mean():.2f}", f"{pickups.std():.2f}")

252.70 144.25


In [2]:
# 从总体中抽样30个
n = 30
sample = np.random.choice(pickups, size=n)

# 计算sample mean,对总体均值做点估计
sample_mean = sample.mean()
print(f"{sample_mean:.2f}")

# measure of accuracy：用sample mean的标准差来衡量   
sample_std = np.std(sample, ddof=1)/np.sqrt(len(sample))
print(f"{sample_std:.2f}")

# 抽样数量达到n=30，已经可以用CLT，用正态分布算置信区间
confidence_in = sample_mean + np.array([-1, 1]) * sample_std * 1.96
print(f"{confidence_in}")

228.07
30.48
[168.31761045 287.81572289]


1. 上面案例中的估计量是样本均值$\bar X_n$。在计算出样本均值后，衡量该估计量的准确性的常用指标：估计量的方差$Var(\bar X_n)$和对应的总体统计量的置信区间都很好计算。
2. 但很多时候the standard error of a estimate是很难直接计算的。比如当估计量是样本中位数的时候，就无法直接得到估计量的方差和置信区间。这时候一种很方便的方法是Bootstrap sample。
2. 虽然可以用大数定律来得到估计量的置信区间。但实践中正态分布的假设并不成立。 ??? 怎么理解

### I.2 Empirical Bootstrap方法
1. 定义：Empirical Bootstrap又称<font color=red>Efron's bootstrap，或nonparametric bootstrap</font>，是一种resampling method。具体方式是：independently sampling with replacement from an existing sample data with same sample size n, and performing inference among these resampled data. 

2. Bootstrap sample过程 \
有抽样样本$\{X_1, X_2, ..., X_n\}$，用它做有放回抽样，每轮抽n个样本$\{X^{*(k)}_1, X^{*(k)}_2, ..., X^{*(k)}_n\}$，一共抽B轮，即$k=(1, 2, ..., B)$。得到的B个bootstrap sample sets: $$\begin{align} 
& \{X^{*(1)}_1, X^{*(1)}_2, ..., X^{*(1)}_n\} \\
& \{X^{*(2)}_1, X^{*(2)}_2, ..., X^{*(2)}_n\}\\
& ...\\
& \{X^{*(B)}_1, X^{*(B)}_2, ..., X^{*(B)}_n\} 
\end{align}$$
<img src="./pics/BootstrapSteps.png" style="zoom:40%">

3. 计算Bootstrap estimates \
以中位数为例：$M_n=median\{X_1, X_2, ..., X_n\}$为原样本中位数。\
从上面的bootstrap sample sets中可以得到：$$\begin{align} 
& M^{*(1)}_n = median\{X^{*(1)}_1, X^{*(1)}_2, ..., X^{*(1)}_n\} \\
& M^{*(2)}_n = median\{X^{*(2)}_1, X^{*(2)}_2, ..., X^{*(2)}_n\}\\
& ...\\
& M^{*(B)}_n = median\{X^{*(B)}_1, X^{*(B)}_2, ..., X^{*(B)}_n\} 
\end{align}$$
用他们可以计算各种Bootstrap estimates: \
(1) <font color=green>用$\hat {Var}_B(M^*_n)$估计</font> <font color=blue>**$Var(M_n)$样本中位数的方差**</font>
$$
\bar M^*_B=\frac{1}{B} {\textstyle \sum_{k=1}^{B}}M^{*(k)}_n \\
\hat {Var}_B(M^*_n) = \frac{1}{B-1} {\textstyle \sum_{k=1}^{B}} (M^{*(k)}_n-\bar M^*_B)^2
$$ 
$\hat {Var}_B(M_n)$称为bootstrap estimate of the variance，也就是样本中位数方差的Bootstrap估计量。\
(2) <font color=green>用$\hat {MSE}_B(M^*_n)$估计</font><font color=blue>**MSE：用样本中位数估总体中位数的MSE**</font>
$$\hat {MSE}_B(M^*_n) = \frac{1}{B} {\textstyle \sum_{k=1}^{B}} (M^{*(k)}_n-M_n)^2
$$
$\hat {MSE}_B(M^*_n)$称为bootstrap estimate of MSE，也就是样本中位数的MSE的Bootstrap估计量。\
(3) <font color=red>**总体中位数**</font><font color=green>的置信水平为$1-\alpha$的置信区间为：</font>
$$\begin{align}
& M_n \pm z_{1-\alpha/2}·\sqrt{\hat {Var}_B(M^*_n)}\\
& z_{1-\alpha/2}是标准正态分布的置信度为（1-\alpha/2）的置信区间的边界\end{align}
$$ 该区间称为bootstrap confidence interval，也就是样本中位数的Bootstrap置信区间。

### I.3 用Empirical Bootstrap处理样例中估计样本中位数的情况

In [3]:
# 计算样本中位数，因此作为总体总位数的估计量
sample_median = np.median(sample)
print(sample_median)

190.5


In [4]:
B = 50
bs_samples = np.zeros((B, n))
# 完成B次Bootstrap sample
for i in range(B):
    bs_sample = np.random.choice(sample, size=n, replace=True)
    bs_samples[i] = bs_sample

In [5]:
# 计算每个Bootstrap sample的sample median，得到Bootstrap medians
bs_medians = np.median(bs_samples, axis=1)

# 衡量估计量的acc: 用Bootstrap medians的标准差作为样本中位数标准差的估计量
var_sample_median = np.var(bs_medians)
print(f"{var_sample_median:.2f}")

# 衡量估计量的acc: 也可以用Bootstrap medians的标准差估计样本中位数的标准差
std_bs_median = np.std(bs_medians, ddof=1) 
print(f"{std_bs_median:.2f}")

5430.41
74.44


In [6]:
# 计算总体中位数的1-α置信区间
confidence_in = sample_median + \
                np.array([-1, 1]) * std_bs_median * 1.96
print(f"{confidence_in}")

[ 44.59860656 336.40139344]


## II. Bootstrap estimates背后的统计原理
#### 1.4.1 $\hat F_n$是F(x)的EDF，也是Bootstrap sample对应的随机变量$X^*$的CDF
1. 原样本${X_1, X_2, ..., X_n}$对应的F(x)的估计量$\hat F_n(x)=\frac{1}{n} {\textstyle \sum_{i=1}^{n}}I(X_i\le x) $，它是基于原样本的EDF。
2. <font color=blue>用原样本${X_1, X_2, ..., X_n}$做Bootstrap sampling，每次得到的值记为随机变量$X^*$的取值。那么$X^*$本质上是以${X_1, X_2, ..., X_n}$为取值空间，每个事件概率$P(X^*=X_i)=\frac{1}{n}$的随机变量。此时，$\hat F_n(x)=\frac{1}{n} {\textstyle \sum_{i=1}^{n}}I(X_i\le x) $是$X^*$的CDF。</font>\
如果抽n个$X^*$，那么就有$$
X^*_1, X^*_2, ..., X^*_n\sim \hat F_n
$$
如果一共抽B轮，每轮抽n次，则有：$$\begin{align} 
& X^{*(1)}_1, X^{*(1)}_2, ..., X^{*(1)}_n \ \sim \hat F_n \\
& X^{*(2)}_1, X^{*(2)}_2, ..., X^{*(2)}_n \ \sim \hat F_n \\
& ...\\
& X^{*(B)}_1, X^{*(B)}_2, ..., X^{*(B)}_n \ \sim \hat F_n 
\end{align}$$
<font color=green>表示：每个Bootstrap sample set都是用分布$\hat F_n$抽样的一组IID样本</font>

#### 1.4.2 统计量的分布函数：中位数为例
1. 基于原样本${X_1, X_2, ..., X_n}$的统计量$M_n$的分布函数：$F_{M_n}(x)=\psi (x;F, n)$ \
原分布是F(x)，原有的抽样样本$X_1, X_2, ..., X_n\sim F$。这些样本的中位数$M_n$的分布取决于F和n，因此可以将样本中位数的分布标记为：$$F_{M_n}(x)=\psi (x;F, n)$$

2. Bootstrap估计量$M^{*(k)}_n$的分布函数：$M^{*}_n=\psi (x;\hat F_n,n)$，<font color=red>这里$\hat F_n$是$M^{*}_n$的CDF</font>。 \
比如Bootstrap median: $$
M^{*(k)}_n = median\{X^{*(k)}_1, X^{*(k)}_2, ..., X^{*(k)}_n\}\\
F_{M^{*(k)}_n}(x) = \psi (x;\hat F_n,n)
$$因为每一组Bootstrap sample都是从相同的分布$\hat F_n$中抽样得到的，所以：$$
\psi (x;\hat F_n,n)=F_{M^{*(1)}_n}(x)=F_{M^{*(2)}_n}(x)=...=F_{M^{*(B)}_n}(x)
$$

#### 1.4.3 估计量的性质分析
1. Bootstrap estimate的CDF与对应的population quantity的CDF相似 \
因为$\hat F_n \overset{P}{\rightarrow} F$，所以当n很大时，$\hat F_n$与F相似。如果$\psi$相对F是smooth的，那么此时$\psi (x;\hat F_n,n)$也会与$\psi (x;F, n)$很相似。\
根据收敛性质：$$ \begin{align}
& \because \hat F_n \overset{P}{\rightarrow} F ,\ \ \psi\ is\ smooth\\
& \therefore F_{M^{*}_n}(x)=\psi (x;\hat F_n,n) \overset{P}{\rightarrow} \psi (x;F, n) = F_{M_n}(x)
\end{align}$$
<font color=orange>这意味着：Bootstrap median $M^{*}_n$的CDF与sample median $M_n$的CDF相似。</font> \
<font color=green>一般化为：Bootstrap estimate的CDF与sample estimate的CDF相似。</font>

2. 当估计量的CDF与对应的总体特征(population quantity)的CDF相似时，可以扩展出新的性质，比如：
<font color=blue>$$\hat {Var}_B(M^*_n) \approx Var(M^{*}_n|\hat F_n)\approx Var(M_n| F) $$</font>
（1）取$Var(M^*_n|\hat F_n)$表示$Var(M^{*}_n)$，加条件是强调它是以$\hat F_n$为CDF的$M^{*}_n$的<font color=red>**总体方差**</font>。 \
（2）取$Var(M_n|F)$表示$Var(M_n)$，是以F为CDF的<font color=red>**总体方差**</font>。 \
（3）取$\hat {Var}_B(M^*_n)= \frac{1}{B-1} {\textstyle \sum_{k=1}^{B}} (M^{*(k)}_n-\bar M^*_B)^2$，表示$M^{*}_n|\hat F_n$对应的B轮Bootstrap抽样得到的<font color=green>**样本方差**</font>。

3. 收敛性证明：\
(1) Bootstrap median的CDF与sample median的CDF相似，所以他们的方差相似\
$\therefore Var(M^{*}_n|\hat F) \approx Var(M_n| F)$<font color=red>——这是两个总体的方差相似</font> \
(2) B可以人为设置为很大的值，B很大时，样本方差和总体方差相似。因为<font color=green>这里$\hat F_n$是$M^{*}_n$的CDF；B越大，抽样轮次越多，样本方差就会逼近总体方差。</font> \
$\therefore \hat {Var}_B(M^*_n) \approx Var(M^{*}_n|\hat F) $ <font color=red>——这是样本方差与总体方差相似</font> \

<font color=blue>**除了variance之外，分位数，偏度、峰度等统计量都有同样的性质。但也不是所有的统计量都适用。**</font>

4. 上述特征对$M_n=min\{X_1, X_2, ..., X_n\}$无效。 \
分析： \
假设${X_1, X_2, ..., X_n} \sim Uni[0, 1]$，有n个样本$\{X_1, X_2, ..., X_n\}$，$M_n=min{X_1, X_2, ..., X_n}$是样本的最小值。\
分析：\
（1）均匀分布的样本最小值服从指数分布。即：$$n·M_n \overset{D}{\rightarrow} Exp(1)$$Exp(1)是参数为1的指数分布。也因此，$M_n$的分布是连续的。\
（2）用原样本做Bootstrap sample。\
· 得到$M^{*}_n$是Bootstrap sample的最小值，即$M^{*}_n=min{X^*_1, X^*_2, ..., X^*_n}$。\
· 因为$P(X^*_i=M_n)=\frac{1}{n}$，因此：$$
P(none\ of X^*_1, X^*_2, ..., X^*_n\ select\ M_n)=(1-\frac{1}{n})^n \approx e^{-1} \\
P(one\ of\ X^*_1, X^*_2, ..., X^*_n select\ M_n)\approx 1-e^{-1} \\
即，P(M^*_n=M_n) \approx 1-e^{-1}
$$可见，$M^*_n$有较大的概率直接取到$M_n$，这也就意味着，$M^*_n$的分布不是指数分布，也就和$M_n$的分布完全不同。

## II. Bootstrap and Statistical Functionals

### II.1 Bootstrap EDF: $\hat F^*_n=\frac{1}{n} {\textstyle \sum_{i=1}^{n}}I(X^*_n\le x) $
**定义：Bootstrap sample的EDF称为Bootstrap EDF**
<font color=orange>$$
\begin{align} 
& F(x) &\overset{抽样}{\rightarrow} \underline{\{X_1, ..., X_n\}} & \Longrightarrow &\underline{\hat{F}_n(x)=\frac{1}{n} {\textstyle \sum_{i=1}^{n}}I(X_n\le x) }\\
& & sample & &EDF  \\
& \{X_1, X_2, ..., X_n\} &\overset{抽样}{\rightarrow} \underline{\{X^*_1, ..., X^*_n\}} & \Longrightarrow &\underline{\hat F^*_n(x)=\frac{1}{n} {\textstyle \sum_{i=1}^{n}}I(X^*_n\le x) }\\ 
& & bootstrap\ sample & & Bootstrap\ EDF\\
\end{align}$$</font>

### II.2 用$\hat F^*_n$作为input的Statistical functional做参数估计

#### II.2.1 估计statistical functional的方式：
取statistical functional为$\theta=T_{target}$，用plug-in方法做估计：
1. 第一种plug-in estimateor：plug-in的对象是EDF
$$\hat {\theta}_n=T_{target}(\hat{F}_n)$$
2. 第二种plug-in estimateor：plug-in的对象是Bootstrap EDF
$$\hat {\theta}^*_n=T_{target}(\hat{F}^*_n)$$

#### II.2.2 两种plug-in estimator的一致性和方差分析
· 假设要估计的对象$\theta$是总体的统计量：$\theta = S(X；F)$，其中$X\sim F(x)$ \
· 有n个样本$\{X_1, ..., X_n\}$，样本统计量$S_n(X_1, X_2, ..., X_n)$是样本的函数，sample estimator是$\hat \theta_n = S_n(X_1, X_2, ..., X_n)$ \
· 做B轮Bootstrap sample，得到B个Bootstrap estimates：
$$\begin{align} 
& \hat {\theta^*}^{(1)}_n = S_n(X^{*(1)}_1, X^{*(1)}_2, ..., X^{*(1)}_n) \\
& \hat {\theta^*}^{(2)}_n = S_n(X^{*(2)}_1, X^{*(2)}_2, ..., X^{*(2)}_n)\\
& ...\\
& \hat {\theta^*}^{(B)}_n = S_n(X^{*(B)}_1, X^{*(B)}_2, ..., X^{*(B)}_n) 
\end{align}$$

**估计量的方差**：可以用以上Bootstrap估计量$\hat{\theta^*}_n$的方差$\hat {Var}_B(\hat{\theta^*}_n)$来作为样本估计量$\hat \theta_n$的方差$Var(\hat \theta_n)$的估计量$$\begin{align}
\bar {\hat {\theta ^*}}_B &  =\frac{1}{B} {\textstyle \sum_{k=1}^{B}}\hat{\theta ^*}^{(k)}_n \\
\hat {Var}_B(\hat{\theta^*}_n)& = \frac{1}{B-1} {\textstyle \sum_{k=1}^{B}} (\hat{\theta^*}^{(k)}_n-\bar {\hat{\theta ^*}}_B)^2
\end{align} $$

前文对中位数方差的分析有结论：<font color=orange>$$\hat {Var}_B(M^*_n) \approx Var(M^{*}_n|\hat F_n)\approx Var(M_n| F) $$</font>
这里也可以证明类似关系：<font color=blue>$$
\hat {Var}_B(\hat {\theta^*}_n) \approx Var(\hat{\theta^*}_n|\hat F_n)\approx Var(\hat \theta_n|F)
$$</font>

1. 证明：$
\hat {Var}_B(\hat {\theta^*}_n) \approx Var(\hat{\theta^*}_n|\hat F_n)$ \
B可以人为设置为很大的值，B很大时，样本方差和总体方差相似。因为<font color=green>这里$\hat F_n$是Bootstrap sample所使用的CDF；B越大，抽样轮次越多，样本方差就会逼近总体方差。</font> ？？？找数学定理依据

2. 证明：$Var(\hat{\theta^*}_n|\hat F_n)\approx Var(\hat \theta_n|F)$ \
符号说明：\
(1)有总体统计量$\theta = T_\theta (F)$，对应的样本统计量是$\hat{\theta}_n=T_\theta(\hat F_n)$。 \
(2)样本统计量的方差$Var(\hat{\theta}_n)=Var(T_\theta(\hat F_n))$

样本统计量$\hat \theta$的方差完全由原分布F(x)和样本数量n决定。因此，可以用函数$\psi(F,n)$


 \\
通常，Var(T_\theta(\hat F_n)) \approx \frac{1}{n}Var(T_\theta(\hat F_1))=\frac{1}{n}Var(T_\theta(F))
$$

因此，Bootstrap variance是variance of estimator $\hat \theta_n$的较好估计。

### II.3 估计准确性